In [5]:
import secrets, base64

Step 1: Generate a secret key

In [12]:
def makeSecret():
    secret = secrets.token_bytes(20)
    base32 = base64.b32encode(secret).decode("utf-8").replace("=","")
    return base32
makeSecret()

'XSG7YLAYSP5GEYFEDETJ4LXMIKFGIOH3'

Step 2: Make a QR code URL

In [ ]:
URI = f"otpauth://totp/<Issuer>:<Account>?secret=<BASE32>&issuer=<Issuer>&algorithm=SHA1&digits=6&period=30"
